In [ ]:
import pandas as pd
import numpy as np
import mne
import mne_bids
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

import q1k_sync_tools as qst

import glob

import warnings
warnings.filterwarnings('ignore')


In [ ]:
project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
task_id_in = "vp"
task_id_out = "vp"
subject_id = '002'
session_id = '02'
run_id = '1'

session_path = project_path + 'sourcefiles/' + subject_id + '_' + session_id[1] + '/' + subject_id + '_' + session_id[1] + '_eeg/'
session_file_name = glob.glob(session_path + '*_' + task_id_in + '_*.mff')

if task_id_out == 'mn':
    event_dict_offset = 1
if task_id_out == 'ap':
    event_dict_offset = 1
if task_id_out == 'vp':
    event_dict_offset = 0
    din_str = ('DIN2','DIN3')

In [ ]:
print(session_path)
print(session_file_name[0])
raw = mne.io.read_raw_egi(session_file_name[0])

In [ ]:
eeg_events = mne.find_events(raw, shortest_event = 1)
#eeg_events = mne.find_events(raw, stim_channel="STI 014", shortest_event = 1)
eeg_event_dict = qst.get_event_dict(raw, eeg_events, event_dict_offset)

In [ ]:
eeg_event_dict

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='Original event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')
py.iplot(fig)


In [ ]:
din_diffs = []
din_diffs_time = []
last_din = 0
# Iterate over the array
for row in eeg_events:
    # Check if the third column corresponds to e1 or e2 values
    if row[2] == eeg_event_dict[din_str[0]] or row[2] == eeg_event_dict[din_str[0]]:
        # Calculate the sequential difference between values in the first column
        if last_din > 0:
            din_diffs.append(row[0] - last_din)
            din_diffs_time.append(row[0])
            last_din = row[0]
        else:
            last_din = row[0]
            #print(last_din)


In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=din_diffs_time, y=din_diffs)
fig.update_layout(title='Time between DIN events')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Inter event interval')

In [ ]:
# handle task specific EEG event interpretation..
eeg_events, eeg_stims, eeg_iti, eeg_din_offset, eeg_event_dict, new_events = qst.eeg_event_test(eeg_events, eeg_event_dict, din_str, task_name=task_id_out)

In [ ]:
column_values = eeg_events[:, 2]

# Get unique values and their counts
unique_values, counts = np.unique(column_values, return_counts=True)

# Print the counts of each unique value
print("Counts of each event ID value:")
for value, count in zip(unique_values, counts):
    print(f"Event ID: {value}, Count: {count}")

# Print the event labels
print("Label\tValue")
for label, value in eeg_event_dict.items():
    print(f"{label}\t{value}")

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='DIN updated event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')

fig.show()

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_stims[1:,0],y=eeg_iti)
fig.update_layout(title='Stim DIN event Inter Trial Intervals (ITI)')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Stim DIBN event ITI (ms)')
fig.show()

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_stims[:,0],y=eeg_din_offset)
fig.update_layout(title='Stim DIN offsets')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Stim DIN offset (ms)')

fig.show()

In [ ]:
fig = raw.plot_sensors(show_names=True)

In [ ]:
# specify power line frequency as required by BIDS
raw.info["line_freq"] = 60

bids_path = mne_bids.BIDSPath(
    subject=subject_id, session=session_id, task=task_id_out, run="1", datatype="eeg", root=project_path
)

print(bids_path)
mne_bids.write_raw_bids(
    raw=raw,
    bids_path=bids_path,
    events=eeg_events,
    event_id=eeg_event_dict,
    format = "EDF",
    overwrite=True,
)

In [ ]:
#!jupyter nbconvert --output {"session_reports/" + bids_path.basename + ".html"} --no-input --to html q1k_eeget_init.ipynb
!jupyter nbconvert --output {"session_reports/" + bids_path.basename + ".html"} --TagRemovePreprocessor.remove_all_outputs_tags='{"exclude"}' --no-input --to html q1k_eeget_init.ipynb
#f'Total of {len(outlier_accum)} outliers out of {all_accounts.size} accounts'

# The following code blocks are for eyelink syncing...

In [ ]:
et = mne.io.read_raw_eyelink ('sourcedata/scott_vep.asc')
df_et = et.to_data_frame()

In [ ]:
fig = px.scatter(df_et,
                x='time',
                y='DIN',
                opacity=.5,
                marginal_y="histogram",
                marginal_x="histogram",
                hover_data={
                        'time',
                        'DIN'})
fig.show()


In [ ]:
input_deriv = np.diff(df_et['DIN'])
deriv_indices = np.where(input_deriv > 1)[0]
deriv_indices_deriv = np.diff(deriv_indices)

In [ ]:
fig = px.scatter(
                y=deriv_indices_deriv,
                #y='DIN',
                opacity=.5,
                marginal_y="histogram",
                marginal_x="histogram")
fig.show()


In [ ]:
fig = px.line(y=deriv_indices_deriv)
fig.show()


In [ ]:
plt.plot(df_et['DIN'])